In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] ="-1"
import torch
import numpy as np
import torch.nn as nn
# import torch.nn.functional as F
# import torchvision

#from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical

from tqdm import tqdm_notebook as tqdm

import json
import nltk
from nltk import word_tokenize

from torch.utils.data.dataloader import default_collate


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
DIC_index_word = {index:word for word, index in DIC_word_index.items()}
word_vectors = np.load("wv_matrix100d.npy")

cuda


In [3]:
# if you don't have money, use this cell

# device = torch.device("cpu")
# print(device)
# DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
# DIC_index_word = {index:word for word, index in DIC_word_index.items()}
# word_vectors = np.load("wv_matrix100d.npy")

In [4]:
sent_len = 10

In [5]:
class InferenceAttModel(torch.nn.Module):
    def __init__(self, latent_dim, seq_length, embed_dim, word_vectors):
        super(InferenceAttModel, self).__init__()
        
        self.hidden_dim = latent_dim        
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        
#         self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
#         self.embedding.load_state_dict({'weight': word_vectors})
#         self.embedding.weight.requires_grad = False
        self.embedding = torch.nn.Embedding.from_pretrained(torch.from_numpy(word_vectors), freeze=True)
#        self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
    
        self.soft = torch.nn.Softmax(-2)
        
        self.RNN1 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, batch_first = True)
        self.RNN2 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, batch_first = True)
        
        self.Projection_layer = torch.nn.Sequential(
            torch.nn.Linear(2 * latent_dim, len(word_vectors)),
        )
        
        self.trainable_W = torch.nn.Sequential(
            torch.nn.Linear(latent_dim ,  latent_dim),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.2),
        )
    def encoder_RNN(self, e_x):
        e_y, hiddens = self.RNN1(e_x)
        
        return e_y, hiddens
    
    def decoder_RNN(self, d_x, hiddens):
        d_y, hiddens2 = self.RNN2(d_x, hiddens)
        return d_y, hiddens2
    
    def forward(self, e_x, ):
        e_x = self.embedding(e_x).float()
        batch_size = e_x.shape[0]
        """
        if(batch_size != 1):
            print(batch_size)
            print("Batch size > 1 is not available now. Batch size please give 1.")
            return
        """
        #print(sent.shape) torch.Size([2, 15, 6087])

        e_output, hiddens = self.encoder_RNN(e_x)
        #attention
        uW = self.trainable_W(e_output)
        
        first = True

        first_index = DIC_word_index['<bos>']
        first_input = torch.tensor([[first_index] * batch_size]).to(device)
        first_input = first_input.view(batch_size, 1, 1)
        for i in range(self.seq_length):
            #embed_dim default:100
            #seq_length = sent_len, default:5
            if(first):
                word = first_input;

                
            #word.shape: [batch_size, 1, 1]
            word = self.embedding(word.view(1, -1)).float();
            #word.shape: [1, batch_size, embed_dim]
            #one for each word, therefore d_output = d_state
            
            
            d_output, hiddens = self.decoder_RNN(word.transpose(1,0), hiddens)
            #d_output.shape: [batch_size, 1, hidden_dim]
            uWv = torch.matmul(uW, d_output.transpose(2,1))
            #uWv.shape: [batch_size, seq_length, 1]
            alpha = self.soft(uWv)

            new_context = e_output.transpose(2,1) @ alpha;
            #e_output.shape [batch_size, seq_length, hidden_dim]
            #alpha.shape [batch_size, seq_length, 1]
            
            #new_context.shape [batch_size, hidden_dim, 1]          
            pre = torch.cat((d_output, new_context.transpose(2,1)), 2)
            ans_logits = self.Projection_layer(pre)
            #ans_index = np.argmax(ans_logits.cpu().detach().numpy())
            
            values, indices = torch.max(ans_logits, -1)
            word = indices;
            #word.shape [batch_size, 1]
            if(first):
                batch_seq = word;
                first = False
            else:
                batch_seq = torch.cat((batch_seq, word), 1)
            
            #ans = DIC_index_word[ans_index]
            #ans_seq.append(ans)
            #word = torch.tensor([ans_index])
        
        #logits = torch.stack(logits, 1)
        return batch_seq

In [6]:
class InferenceBiAttModel(torch.nn.Module):
    def __init__(self, latent_dim, seq_length, embed_dim, word_vectors):
        super(InferenceBiAttModel, self).__init__()
        
        self.hidden_dim = latent_dim        
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        
#         self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
#         self.embedding.load_state_dict({'weight': word_vectors})
#         self.embedding.weight.requires_grad = False
        self.embedding = torch.nn.Embedding.from_pretrained(torch.from_numpy(word_vectors), freeze=True)
#        self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
    
        self.soft = torch.nn.Softmax(-2)
        
        self.RNN1 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, bidirectional = True, batch_first = True)
        self.RNN2 = torch.nn.LSTM(self.embed_dim, latent_dim * 2,  num_layers=2, batch_first = True)
        
        self.Projection_layer = torch.nn.Sequential(
            torch.nn.Linear(4 * latent_dim, len(word_vectors)),
        )
        
        self.trainable_W = torch.nn.Sequential(
            torch.nn.Linear(2 * latent_dim , 2 * latent_dim),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.2),
        )
    def encoder_RNN(self, e_x):
        e_y, hiddens = self.RNN1(e_x)
        
        return e_y, hiddens
    
    def decoder_RNN(self, d_x, hiddens):
        d_y, hiddens2 = self.RNN2(d_x, hiddens)
        return d_y, hiddens2
    
    def forward(self, e_x):
        e_x = self.embedding(e_x).float()
        batch_size = e_x.shape[0]
        
        #d_x = self.embedding(d_x).float()

        #print(sent.shape) torch.Size([2, 15, 6087])

        e_output, hiddens = self.encoder_RNN(e_x)
        
        #attention
        uW = self.trainable_W(e_output)
        hiddens = [hiddens[0].view(int(hiddens[0].shape[0]/2), hiddens[0].shape[1], hiddens[0].shape[2]*2), hiddens[1].view(int(hiddens[1].shape[0]/2), hiddens[1].shape[1], hiddens[1].shape[2]*2)]
        
#         probs = 1.0;
#         linear_decay_rate = 0.1;
        
        first = True
        pre = None
        
        
        
        first_index = DIC_word_index['<bos>']
        first_input = torch.tensor([[first_index] * batch_size]).to(device)
        first_input = first_input.view(batch_size, 1, 1)
        for i in range(self.seq_length):
            #embed_dim default:100
            #seq_length = sent_len, default:5
            if(first):
                word = first_input;

                
            #word.shape: [batch_size, 1, 1]
            word = self.embedding(word.view(1, -1)).float();
            #word.shape: [1, batch_size, embed_dim]
            #one for each word, therefore d_output = d_state
            
            
            d_output, hiddens = self.decoder_RNN(word.transpose(1,0), hiddens)
            #d_output.shape: [batch_size, 1, hidden_dim]
            uWv = torch.matmul(uW, d_output.transpose(2,1))
            #uWv.shape: [batch_size, seq_length, 1]
            alpha = self.soft(uWv)

            new_context = e_output.transpose(2,1) @ alpha;
            #e_output.shape [batch_size, seq_length, hidden_dim]
            #alpha.shape [batch_size, seq_length, 1]
            
            #new_context.shape [batch_size, hidden_dim, 1]          
            pre = torch.cat((d_output, new_context.transpose(2,1)), 2)
            ans_logits = self.Projection_layer(pre)
            #ans_index = np.argmax(ans_logits.cpu().detach().numpy())
            
            values, indices = torch.max(ans_logits, -1)
            word = indices;
            #word.shape [batch_size, 1]
            if(first):
                batch_seq = word;
                first = False
            else:
                batch_seq = torch.cat((batch_seq, word), 1)
            
            #ans = DIC_index_word[ans_index]
            #ans_seq.append(ans)
            #word = torch.tensor([ans_index])
        
        #logits = torch.stack(logits, 1)
        return batch_seq

In [7]:
#
def Inference(model_path, question_path):
    def pad_post_zero(a, length):
        ret = []
        for _list in a:
            if(len(_list) < length):
                for ct in range(len(_list),length,1):
                    _list.append(DIC_word_index["<pad>"])
            if(len(_list) > length):
                _list = _list[:length]

            ret.append(_list)
        return ret
    def my_collate(datas):
        batch = {}
        batch['encoder_x'] = torch.tensor([data for data in datas])
        return batch
    latent_dim = 512
    embed_dim = word_vectors.shape[1]
    batch_size = 1024
    
    model = InferenceBiAttModel(latent_dim, sent_len, embed_dim, word_vectors)
    #saved_model = torch.load("models/Model92")
    saved_model = torch.load(model_path)
    model.load_state_dict(saved_model['model'])
    model = model.to(device)
    model.eval()
    
    encode_x = []
    #with open('sel_conversation/question.txt', 'r') as f:
    with open(question_path, 'r') as f:
        for l in f:
            l = l.split()
            tmp = []
            for word in l:
                try:
                    index = DIC_word_index[word]
                except KeyError:
                    index = DIC_word_index['<unk>']
                tmp.append(index)
            encode_x.append([DIC_word_index['<bos>']] + tmp + [DIC_word_index['<eos>']])        
    encode_x = pad_post_zero(encode_x, sent_len)
    
    #print(np.array(encode_x).shape); (len(encode_x, sent_len)
    dataloader = torch.utils.data.DataLoader(encode_x, batch_size = batch_size, shuffle = False, collate_fn = my_collate)
    ANSWERS = list()
    for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        with torch.no_grad():
            e_x = batch['encoder_x'].long().to(device)
        batch_seq = model.forward(e_x)
        for seq in batch_seq:
            #seq shape: [sent_len]
            ans = []
            for ans_index in seq:
                ans.append(DIC_index_word[ans_index.item()])
            ANSWERS.append(ans)
    return(ANSWERS)
    """
    ANSWERS = list()
    for index in range(5):
        e_x = torch.tensor([encode_x[index]]).long().to(device)
        ans = model.forward(e_x)
        ANSWERS.append(ans)
    
    return(ANSWERS)
    """

In [8]:
ANSWERS = Inference(model_path="models/Model103", question_path='mlds_hw2_2_data/test_input.txt')

In [9]:
ANSWERS 

[['我',
  '的',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['來',
  '紀念',
  '退伍軍人',
  '節',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['責任', '更大', '纔', '只好', '司法', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['我們', '必須', '把', '這個', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['我們', '還會', '有', '其他', '的', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['我們',
  '的',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['我',
  ',',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['最', '美好', '的', '夜晚', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['他',
  '的',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['我',
  '通報',
  '通報',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['我們', '得', '出動', '出動', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos

In [10]:
print(len(ANSWERS))
# fo = open("answer.txt","w")
# for line in ANSWERS:
#     fo.write(line)
# #fo.write(ANSWERS)

10000


In [11]:
with open('answer.txt', 'w') as f:
    for res in tqdm(ANSWERS):
        try:
            end = res.index('<eos>')
        except ValueError:
            end = len(res)
        sent = "".join(res[:end])
        f.write(sent+'\n')
    